In [1]:
!pip install pyura
!pip install geopy

# Import packages
import datetime
import math
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
import time
import pickle
import zipfile
import requests
import io
from datetime import date
from pyura import Client
from requests import Session
from geopy.geocoders import Nominatim
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from pathlib import Path
import shapefile
import os
import shutil


# URA Access Key
access_key = '8c04ea68-08ad-4e37-ae9d-27a704d979b6'


# 1. Get Data from all sources / api

In [2]:
def get_json_from_zip_url(json_name, zip_file_url):
    r = requests.get(zip_file_url, stream=True)
    with zipfile.ZipFile(io.BytesIO(r.content)) as myzip:
        with myzip.open(json_name) as myfile:
            data = json.load(myfile)
            return data
    return None

def get_df_from_url(xl_url):
    return pd.read_csv(xl_url)

json_name = 'hawker-centres-geojson.geojson'
zip_file_url = 'https://data.gov.sg/dataset/aeaf4704-5be1-4b33-993d-c70d8dcc943e/download'
data = get_json_from_zip_url(json_name, zip_file_url)

In [3]:
hawker_dict_data_gov = {}
for i in range(len(data['features'])):
    string = (data['features'][i]['properties']['Description'])
    list_of_strings = string.split()
    postal_code = list_of_strings[list_of_strings.index('<th>ADDRESSPOSTALCODE</th>') + 1][4:6]
    address = list_of_strings[list_of_strings.index('<th>NAME</th>') + 1 : list_of_strings.index('<th>ADDRESSTYPE</th>') - 2]
    hawker_add = " ".join(address)[4:-5]
    hawker_dict_data_gov[postal_code] = hawker_add
    
hawker_dict_data_gov

{'76': 'Yishun Ring Road Blk 104/105 (Chong Pang Market and Food Centre)',
 '61': 'Taman Jurong Market and Food Centre',
 '38': 'Aljunied Ave 2 Blk 117 (Blk 117 Aljunied Market and Food Centre)',
 '15': 'Jalan Bukit Merah Blk 6 (ABC Brickworks Market/Food Centre)',
 '52': 'Our Tampines Hub',
 '60': 'Jurong East Street 24 Blk 254 (Yuhua Village Market and Food Centre)',
 '27': 'Ghim Moh Road Blk 20',
 '57': 'Sembawang Hills Food Centre (Jalan Leban Food Centre)',
 '05': "New Market Road Blk 32 (People's Park Food Centre)",
 '14': 'Mei Chin Road Blk 159 (Mei Chin Road Market)',
 '08': 'Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Pagar Plaza Market and Food Centre)',
 '10': 'Telok Blangah Drive Blk 79 (Telok Blangah Food Centre)',
 '16': 'Havelock Road Blk 22A/B (Havelock Road Cooked Food Centre)',
 '46': 'East Coast Lagoon Food Village',
 '33': 'Geylang Bahru Blk 69 (Blk 69 Geylang Bahru Market and Food Centre)',
 '64': 'Jurong West Street 52 Blk 505',
 '67': 'Bukit Panjang Hawker Centre',


# 1.2 Get 5Y Data from URA

In [4]:
def ura_to_df(json_data):
    main_df = pd.DataFrame()
    for i in range(len(json_data)):
        trans_list = json_data[i]
        df = pd.DataFrame(trans_list)
        formatted_df = pd.concat([df.drop('transaction', axis=1), pd.DataFrame(df['transaction'].tolist())], axis=1)
        # Append to df 
        main_df = main_df.append(formatted_df)
    return main_df

def query_ura(access_key):
    main_df = pd.DataFrame()
    # Gen Daily Tokenb
    sess = Session()
    response = sess.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers={"AccessKey": access_key})
    token = response.json()["Result"]
    # GET PRIVATE RESI DATA IN BATCHES (4) 
    for batch in range(1,5):
        print(f"Querying URA API Batch {batch}/4")
        out = sess.get("https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction", params={"batch": batch}, headers={"AccessKey": access_key, "Token": token})
        response_json = out.json()
        if "Result" in response_json:
            response_data = response_json["Result"]
        else:
            response_data = response_json
        formatted_df = ura_to_df(response_data)
        main_df = main_df.append(formatted_df)
    main_df = main_df.reset_index(drop=True)
    return main_df

In [5]:
sess = Session()
response = sess.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers={"AccessKey": access_key})

ura_df = query_ura(access_key)
#ura_df.to_csv('5y_ura.csv')

Querying URA API Batch 1/4
Querying URA API Batch 2/4
Querying URA API Batch 3/4
Querying URA API Batch 4/4


# 1.3 Get all amenities in each district

In [31]:
# Global path for the scripts
path_env = os.path.dirname(os.path.abspath("Combining Datasets v2.ipynb"))

# Common paths used by multiple scripts
path_chromedriver = str(Path(path_env, 'chromedriver.exe'))
path_download = os.path.join(Path.home(), "Downloads")

# Map to Specific Location in the District for OneMap Search
district_mapping = {
    "1" : ["RAFFLES PLACE MRT STATION EXIT A", "CECIL BUILDING", "MARINA BAY SINGAPORE", "PEOPLE'S PARK COMPLEX"],
    "2" : ["ANSON ROAD", "TANJONG PAGAR MRT STATION EXIT A"],
    "3" : ["QUEENSTOWN MRT STATION EXIT A", "TIONG BAHRU MRT STATION EXIT A"],
    "4" : ["TELOK BLANGAH RISE MARKET", "HARBOURFRONT MRT STATION EXIT A"],
    "5" : ["PASIR PANJANG MRT STATION EXIT A", "CLEMENTI MRT STATION EXIT A"],
    "6" : ["HIGH STREET PLAZA", "CALTEX BEACH ROAD"],
    "7" : ["DBS MIDDLE ROAD 210", "GOLDEN MILE COMPLEX"],
    "8" : ["LITTLE INDIA MRT STATION EXIT A"],
    "9" : ["ORCHARD MRT STATION EXIT A", "THE CAIRNHILL", "GREAT WORLD MRT STATION"],
    "10" : ["BUKIT TIMAH ROAD", "HOLLAND VILLAGE MRT STATION", "TANGLIN MALL"],
    "11" : ["WATTEN ESTATE", "NOVENA MRT STATION EXIT A", "THOMSON PLAZA"],
    "12" : ["BALESTIER PLAZA", "TOA PAYOH MRT STATION EXIT A", "SERANGOON MRT STATION EXIT A"],
    "13" : ["MACPHERSON MRT STATION EXIT A", "BRADDELL MRT STATION EXIT A"],
    "14" : ["PAYA LEBAR MRT STATION EXIT A", "EUNOS MRT STATION EXIT A"],
    "15" : ["TANJONG KATONG GIRLS' SCHOOL", "KOON SENG PARK", "TANJONG KATONG MRT STATION EXIT A"],
    "16" : ["BEDOK MRT STATION EXIT A", "TANAH MERAH MRT STATION EXIT A", "SUNGEI BEDOK MRT STATION", "KEW DRIVE PLAYGROUND"],
    "17" : ["MARIAM WAY PLAYGROUND", "KEMBANGAN MRT STATION EXIT A"],
    "18" : ["TAMPINES MRT STATION EXIT A", "PASIR RIS MRT STATIONEXIT A"],
    "19" : ["SERANGOON GARDEN MARKET EXIT A", "HOUGANG MRT STATION EXIT A", "PUNGGOL MRT STATION EXIT A"],
    "20" : ["BISHAN MRT STATION EXIT A", "ANG MO KIO MRT STATION EXIT A"],
    "21" : ["ESSO UPPER BUKIT TIMAH A", "CLEMENTI PARK", "ULU PANDAN COMMUNITY CLUB"],
    "22" : ["JURONG EAST MRT STATION EXIT A", "BOON LAY MRT STATION EXIT A", "LAKESIDE MRT STATION EXIT A", "CHINESE GARDEN MRT STATION EXIT A"],
    "23" : ["HILLVIEW MRT STATION EXIT A", "GERMAN EUROPEAN SCHOOL SINGAPORE", "BUKIT PANJANG MRT STATION EXIT A1", "CHOA CHU KANG MRT STATION EXIT A"],
    "24" : ["LIM CHU KANG CAMP I", "GARDEN VALE @ TENGAH"],
    "25" : ["KRANJI MRT STATION EXIT A", "THE WOODGROVE"],
    "26" : ["UPPER THOMSON MRT STATION", "SPRINGLEAF GARDEN"],
    "27" : ["YISHUN MRT STATION EXIT A", "SEMBAWANG MRT STATION EXIT A"],
    "28" : ["THE SELETAR MALL"]
}

district_postal = {
    "1" : ["01", "02", "03", "04", "05", "06"],
    "2" : ["07", "08"],
    "3" : ["14", "15", "16"],
    "4" : ["09", "10"],
    "5" : ["11", "12", '13'],
    "6" : ["17"],
    "7" : ["18", "19"],
    "8" : ["20", "21"],
    "9" : ["22", "23"],
    "10" : ["24", "25", "26", "27"],
    "11" : ["28", "29", "30"],
    "12" : ["31", "32", "33"],
    "13" : ["34", "35", "36", "37"],
    "14" : ["38", "39", "40", "41"],
    "15" : ["42", "43", "44", "45"],
    "16" : ["46", "47", "48"],
    "17" : ["49", "50", "81"],
    "18" : ["51", "52"],
    "19" : ["53", "54", "55", "82"],
    "20" : ["56", "57"],
    "21" : ["58", "59"],
    "22" : ["60", "61", "62", "63", "64"],
    "23" : ["65", "66", "67", "68"],
    "24" : ["69", "70", "71"],
    "25" : ["72", "73"],
    "26" : ["77", "78"],
    "27" : ["75", "76"],
    "28" : ["79", "80"]
}



def get_dict_data(address, driver):
    # address is list
    ans = dict()
    count = 0
    # Just download shapefiles (Only need run once / Any address)
    download_all_shp(address[0], driver)
    os.chdir(path_download)
    for add in address:
        count += 1
        print(f"{add}, {count}/{len(address)}")
        school = download_all_schools(str(add), driver)
        inter = dict()
        inter['school'] = school
        for i in ['hawkercentre', 'supermarkets']:
            try:
                with zipfile.ZipFile(f"{i}.zip","r") as zip_ref:
                    zip_ref.extractall(f"{i}")
                sf = shapefile.Reader(f"{i}/{i}/{i.upper()}.shp")
                output = sf.__geo_interface__
                inter[i] = output['features']
                sf.close()
            except:
                inter[i] = None 
        ans[add] = inter
    try:
        for i in ['hawkercentre', 'supermarkets']:
            os.remove(f"{i}.zip")
            shutil.rmtree(f"{i}")
    except FileNotFoundError:
        pass
    os.chdir(path_env)
    return ans

def download_all_shp(address, driver):
    url = 'https://www.onemap.sg'
    driver.get(url)
    try:
        driver.find_element_by_xpath('//*[@id="my-text"]/img').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="search-text"]').clear()
        query = driver.find_element_by_xpath('//*[@id="search-text"]')
    except:
        try:
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="search-text"]').clear()
            query = driver.find_element_by_xpath('//*[@id="search-text"]')
        except:
            pass
    query.send_keys(address)
    time.sleep(5)
    query.send_keys(Keys.ENTER)
    # Essential Amenities
    driver.find_element_by_xpath('//*[@id="EssDisplay"]').click()
    time.sleep(1)
    for i in [1, 2]:
        driver.find_element_by_xpath(f'//*[@id="mCSB_4_container"]/div/div[{str(i)}]/div/div[2]/label').click()
        select = Select(driver.find_element_by_xpath('/html/body/div[2]/div/select'))
        # select by value 
        select.select_by_value('shp')
        driver.find_element_by_xpath('/html/body/div[2]/div/div[10]/button[2]').click()
        driver.find_element_by_xpath('/html/body/div[2]/div/div[10]/button[1]').click()
    return None

def download_all_schools(address, driver):
    url = 'https://www.onemap.sg'
    driver.get(url)
    try:
        driver.find_element_by_xpath('//*[@id="my-text"]/img').click()
    except:
        pass
    try:
        driver.find_element_by_xpath('//*[@id="search-text"]').clear()
        query = driver.find_element_by_xpath('//*[@id="search-text"]')
    except:
        try:
            time.sleep(5)
            driver.find_element_by_xpath('//*[@id="search-text"]').clear()
            query = driver.find_element_by_xpath('//*[@id="search-text"]')
        except:
            pass
    query.send_keys(address)
    time.sleep(5)
    query.send_keys(Keys.ENTER)
    # Essential Amenities
    driver.find_element_by_xpath('//*[@id="EssDisplay"]').click()
    time.sleep(1)
    try:
        #Schools
        driver.find_element_by_xpath('//*[@id="SQDisplay"]').click()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="schoolMenu"]/button[2]').click()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="AgreeBoxSQ"]/label').click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        schools = [school.text for school in soup.find_all('span', class_='school-header ng-binding')]
    except Exception as e:
        print(e)
        schools = None
    return schools

# Get distance between 2 locations using claims distance finder https://pacgov.agd.gov.sg/ipac/html/W51RouteDistanceFinder.html
def get_distance(start, end, driver):
    url = 'https://pacgov.agd.gov.sg/ipac/html/W51RouteDistanceFinder.html'
    driver.get(url)
    driver.find_element_by_xpath('//*[@id="txtSearchFromText"]').send_keys(str(start))
    driver.find_element_by_xpath('//*[@id="txtSearchToText"]').send_keys(str(end))
    driver.find_element_by_xpath('/html/body/form/table[1]/tbody/tr[12]/td[3]/input').click()
    driver.find_element_by_xpath('//*[@id="divResults"]/table/tbody/tr[2]/td/div').click()
    driver.find_element_by_xpath('/html/body/form/table[1]/tbody/tr[14]/td[3]/input').click()
    driver.find_element_by_xpath('//*[@id="divResults"]/table/tbody/tr[2]/td/div').click()
    driver.find_element_by_xpath('/html/body/form/table[1]/tbody/tr[16]/td/input').click()
    
def get_2km_nearby(list_of_address, district_mapping, district_postal, hawker_gov):
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    final = get_dict_data(list_of_address, driver)
    main_df = pd.DataFrame()
    for address, contents in final.items():
        new_dict = {}
        new_dict['address'] = address
        for key, value in district_mapping.items():
            if address in value:
                f_key = key
        postal_codes = district_postal[f_key]
        for tag, content in contents.items():
            if tag == 'school':
                new_dict[tag] = content
            else:
                # Form new list 
                all_names = []
                # From SHP Downloaded
                if content is not None:
                    for place in content:
                        if tag == 'hawkercentre':
                            if place['properties']['ADDRESSPOS'][:2] in postal_codes:
                                name = place['properties']['NAME']
                                if name in all_names:
                                    continue
                                else:
                                    all_names.append(name)
                        elif tag == 'supermarkets':
                            if place['properties']['POSTCODE'][:2] in postal_codes:
                                name = place['properties']['LIC_NAME']
                                if name in all_names:
                                    continue
                                else:
                                    all_names.append(name)
                # Hawker Centres from Data.gov
                if tag == 'hawkercentre':
                    for keys in hawker_gov.keys():
                        if keys in postal_codes:
                            if hawker_gov[keys] not in all_names: 
                                all_names.append(hawker_gov[keys])
                new_dict[tag] = all_names
        data = [[new_dict['address'], new_dict['school'], new_dict['hawkercentre'], new_dict['supermarkets']]]
        df = pd.DataFrame(data, columns = ['address', 'school', 'hawkercentre', 'supermarkets'])
        main_df = main_df.append(df)
    driver.close()
    return main_df

In [7]:
flat_list = [item for sublist in district_mapping.values() for item in sublist]
nearby_df = get_2km_nearby(flat_list, district_mapping, district_postal, hawker_dict_data_gov)
nearby_df.head()

RAFFLES PLACE MRT STATION EXIT A, 1/69
CECIL BUILDING, 2/69
MARINA BAY SINGAPORE, 3/69
PEOPLE'S PARK COMPLEX, 4/69
ANSON ROAD, 5/69
TANJONG PAGAR MRT STATION EXIT A, 6/69
QUEENSTOWN MRT STATION EXIT A, 7/69
TIONG BAHRU MRT STATION EXIT A, 8/69
TELOK BLANGAH RISE MARKET, 9/69
HARBOURFRONT MRT STATION EXIT A, 10/69
PASIR PANJANG MRT STATION EXIT A, 11/69
CLEMENTI MRT STATION EXIT A, 12/69
HIGH STREET PLAZA, 13/69
CALTEX BEACH ROAD, 14/69
DBS MIDDLE ROAD 210, 15/69
GOLDEN MILE COMPLEX, 16/69
LITTLE INDIA MRT STATION EXIT A, 17/69
ORCHARD MRT STATION EXIT A, 18/69
THE CAIRNHILL, 19/69
GREAT WORLD MRT STATION, 20/69
BUKIT TIMAH ROAD, 21/69
HOLLAND VILLAGE MRT STATION, 22/69
TANGLIN MALL, 23/69
WATTEN ESTATE, 24/69
NOVENA MRT STATION EXIT A, 25/69
THOMSON PLAZA, 26/69
BALESTIER PLAZA, 27/69
TOA PAYOH MRT STATION EXIT A, 28/69
SERANGOON MRT STATION EXIT A, 29/69
MACPHERSON MRT STATION EXIT A, 30/69
BRADDELL MRT STATION EXIT A, 31/69
PAYA LEBAR MRT STATION EXIT A, 32/69
EUNOS MRT STATION EXIT 

,address,school,hawkercentre,supermarkets
0,RAFFLES PLACE MRT STATION EXIT A,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BENDEMEER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,CECIL BUILDING,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BUKIT MER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,MARINA BAY SINGAPORE,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BENDEMEER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,PEOPLE'S PARK COMPLEX,"[ANGLO-CHINESE SCHOOL (BARKER ROAD), BENDEMEER...",[Smith Street Blk 335 (Chinatown Complex Marke...,[]
0,ANSON ROAD,"[BUKIT MERAH SECONDARY SCHOOL, CHIJ ST. THERES...",[Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Paga...,[]


In [8]:
ameneties_df = pd.DataFrame()
for key, value in district_mapping.items():
    school_list = []
    hawker_list = []
    market_list = []
    for address in value:
        try:
            school_list += nearby_df.loc[nearby_df.address == address, 'school'].values[0]
        except TypeError:
            continue
        try:
            hawker_list += nearby_df.loc[nearby_df.address == address, 'hawkercentre'].values[0]
        except TypeError:
            continue
        try:
            market_list += nearby_df.loc[nearby_df.address == address, 'supermarkets'].values[0]
        except TypeError:
            continue
    data = [[key, list(set(school_list)), list(set(hawker_list)), list(set(market_list))]]
    df = pd.DataFrame(data, columns = ['district', 'school', 'hawkercentre', 'supermarkets'])
    ameneties_df = ameneties_df.append(df)
ameneties_df.head()

,district,school,hawkercentre,supermarkets
0,1,"[OUTRAM SECONDARY SCHOOL, BENDEMEER SECONDARY ...",[Amoy Street Food Centre (Telok Ayer Food Cent...,[]
0,2,"[OUTRAM SECONDARY SCHOOL, CRESCENT GIRLS' SCHO...",[Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Paga...,[]
0,3,"[OUTRAM SECONDARY SCHOOL, ST. MARGARET'S SECON...","[Commonwealth Crescent Market, Bukit Merah Lan...","[NTUC FAIRPRICE CO-OPERATIVE LTD, COLD STORAGE..."
0,4,"[OUTRAM SECONDARY SCHOOL, QUEENSWAY SECONDARY ...",[Telok Blangah Drive Blk 79 (Telok Blangah Foo...,"[THE QUALITYMART DEPOT PTE. LTD., NTUC FAIRPRI..."
0,5,"[ST. MARGARET'S SECONDARY SCHOOL, NUS HIGH SCH...","[Clementi West Street 2 Blk 726, West Coast Dr...","[NTUC FAIRPRICE CO-OPERATIVE LTD, TANESEI TRAD..."


In [9]:
ameneties_df.to_csv('ameneties_in_district.csv')

In [11]:
ameneties_df = pd.read_csv('ameneties_in_district.csv', index_col=0)

# 1.4 Append Geo Coords for Transactions

In [6]:
hdb_df = pd.read_csv('https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/resale_flat_prices_based_on_registration_date_from_jan_2017_onwards.csv', index_col=0)

In [7]:
# FOR HDB DATAFRAME

def replace_words(x, words):
    s = x['street']
    for k, v in words.items():
        if k in s:
            # Just for road - 
            if (k == 'RD') and ("ORD" in s):
                break
            s = s.replace(k, v)
    x['street'] = s
    return x

naming_mapping = {
    'CTRL' : 'CENTRAL',
    'UPP': 'UPPER',
    'JLN': 'JALAN',
    'TG': 'TANJONG',
    'BT': 'BUKIT',
    'NTH': 'NORTH',
    'LOR': 'LORONG', 
    'RD' : 'ROAD',
    'KG' : 'KAMPONG',
    'C\'WEALTH' : 'COMMONWEALTH'
}

hdb_df['street'] = hdb_df['street_name']
hdb_df = hdb_df.apply(lambda x: replace_words(x, naming_mapping), axis=1)

In [22]:
list_streets = list(ura_df.street.unique()) + list(hdb_df.street.unique())

In [4]:
list_streets = list(hdb_df.street.unique())

In [28]:
list_streets

['ZEHNDER ROAD',
 'NEO PEE TECK LANE',
 'COVE DRIVE',
 'BUKIT TERESA ROAD',
 'ALEXANDRA VIEW',
 'YONG SIAK STREET',
 'BEACH ROAD',
 'FABER AVENUE',
 'OCEAN DRIVE',
 'PASIR PANJANG ROAD',
 'CENTRAL BOULEVARD',
 'SPOTTISWOODE PARK ROAD',
 'TREASURE ISLAND',
 'TELOK BLANGAH ROAD',
 'SHENTON WAY',
 'JALAN MAT JAMBOL',
 'CLEMENTI AVENUE 1',
 'BENCOOLEN STREET',
 'MOUNT FABER ROAD',
 'CHWEE CHIAN VIEW',
 'WEST COAST CRESCENT',
 'FABER GREEN',
 'DOVER RISE',
 'WEST COAST WAY',
 'PEARL BANK',
 'YEW SIANG ROAD',
 'PRINCE CHARLES CRESCENT',
 'PRINSEP LINK',
 'FABER GROVE',
 'WEST COAST VIEW',
 'LAKESHORE VIEW',
 'ENGGOR STREET',
 'MIDDLE ROAD',
 'PASIR PANJANG HILL',
 'MARINA BOULEVARD',
 'SOUTH BRIDGE ROAD',
 'GOPENG STREET',
 'WEST COAST GROVE',
 'FABER TERRACE',
 'NEIL ROAD',
 'CHIN SWEE ROAD',
 'ROBINSON ROAD',
 'HAVELOCK ROAD',
 'QUEENSWAY',
 'TAN QUEE LAN STREET',
 'WEST COAST RISE',
 'DUNDEE ROAD',
 'SOUTH BUONA VISTA ROAD',
 'LOWER DELTA ROAD',
 'BUKIT TERESA CLOSE',
 'ANSON ROAD',
 'UPP

In [ ]:
# Slower way but cfm can run

import requests

lat_long_mapping = {}
# User Agent can be anything 
count = 0
invalid_count = 0
total = len(list_streets)
for street in list_streets:
    print(f"{count} / {total} , {street}")
    url_response = 0
    while url_response != 200:
        try:
            url = requests.get(f'https://nominatim.openstreetmap.org/search?country=Singapore&street={street}&format=json&limit=1')
            url_response = url.status_code
            continue
        except:
            break
    try:
        add_json = url.json()
        lat, long = float(add_json[0]['lat']), float(add_json[0]['lon']) 
        lat_long_mapping[street] = [lat, long]
        count += 1
    except:
        invalid_count += 1
        print(f"{street}, {count} / {total}")
        continue
    print(lat_long_mapping)
print(invalid_count)

0 / 2222 , ZEHNDER ROAD
{'ZEHNDER ROAD': [1.2821298, 103.7868785]}
1 / 2222 , NEO PEE TECK LANE
{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915]}
2 / 2222 , COVE DRIVE
{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866]}
3 / 2222 , BUKIT TERESA ROAD
{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406]}
4 / 2222 , ALEXANDRA VIEW
{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768]}
5 / 2222 , YONG SIAK STREET
{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALE

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888]}
20 / 2222 , WEST COAST CRESCENT
{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.29204

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

...

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

{'ZEHNDER ROAD': [1.2821298, 103.7868785], 'NEO PEE TECK LANE': [1.2920468, 103.7685915], 'COVE DRIVE': [1.2442085, 103.8274866], 'BUKIT TERESA ROAD': [1.2754536, 103.8281406], 'ALEXANDRA VIEW': [1.2902687, 103.8177768], 'YONG SIAK STREET': [1.2824613, 103.8304847], 'BEACH ROAD': [1.3044356, 103.8662427], 'FABER AVENUE': [1.3044356, 103.8662427], 'OCEAN DRIVE': [1.24946, 103.8413371], 'PASIR PANJANG ROAD': [1.2918901, 103.7690402], 'CENTRAL BOULEVARD': [1.2740985, 103.8609659], 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071], 'TREASURE ISLAND': [1.2487983, 103.8432072], 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922], 'SHENTON WAY': [1.2782142, 103.8495062], 'JALAN MAT JAMBOL': [1.2782142, 103.8495062], 'CLEMENTI AVENUE 1': [1.2782142, 103.8495062], 'BENCOOLEN STREET': [1.2782142, 103.8495062], 'MOUNT FABER ROAD': [1.2782142, 103.8495062], 'CHWEE CHIAN VIEW': [1.279643, 103.7888], 'WEST COAST CRESCENT': [1.298743, 103.7653247], 'FABER GREEN': [1.31862, 103.7583806], 'DOVER RISE': [1.

In [5]:
"""
# Faster way but API Server might be overloaded and cant run, if cant run please use existing pickle file 
lat_long_mapping = {}
# User Agent can be anything 
geolocator = Nominatim(user_agent="testtest321")
count = 0
invalid_count = 0
total = len(list_streets)
for street in list_streets:
    try:
        print(f"{count} / {total}")
        location = geolocator.geocode({"street": street, "country": "Singapore"})
        lat_long_mapping[street] = [location.latitude, location.longitude]
        count += 1
        #print(f"Qeurying {count} / {total} unique addresses from Nominatim API")
    except AttributeError:
        invalid_count += 1
        print(f"{street}, {count} / {total}")
        continue
print(invalid_count)
"""

0 / 550


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?street=ANG+MO+KIO+AVE+10&country=Singapore&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [ ]:
# Save to pickle file for reuse next time
with open('lat_long_mapping.pickle', 'wb') as handle:
    pickle.dump(lat_long_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
# Load Existing pickle file for mapping, or run one of the above functions
with open('lat_long_mapping.pickle', 'rb') as handle:
    lat_long_mapping = pickle.load(handle)
lat_long_mapping

{'ZEHNDER ROAD': [1.2821298, 103.7868785],
 'NEO PEE TECK LANE': [1.2920468, 103.7685915],
 'COVE DRIVE': [1.2442085, 103.8274866],
 'BUKIT TERESA ROAD': [1.2754536, 103.8281406],
 'ALEXANDRA VIEW': [1.2902687, 103.8177768],
 'YONG SIAK STREET': [1.2824613, 103.8304847],
 'BEACH ROAD': [1.3044356, 103.8662427],
 'FABER AVENUE': [1.3191116, 103.7591688],
 'OCEAN DRIVE': [1.24946, 103.8413371],
 'PASIR PANJANG ROAD': [1.2918901, 103.7690402],
 'CENTRAL BOULEVARD': [1.2740985, 103.8609659],
 'SPOTTISWOODE PARK ROAD': [1.2734521, 103.8394071],
 'TREASURE ISLAND': [1.2487983, 103.8432072],
 'TELOK BLANGAH ROAD': [1.2658126, 103.8209922],
 'SHENTON WAY': [1.2782142, 103.8495062],
 'JALAN MAT JAMBOL': [1.2784116, 103.7919099],
 'CLEMENTI AVENUE 1': [1.3075943, 103.7691926],
 'BENCOOLEN STREET': [1.2993745, 103.8507974],
 'MOUNT FABER ROAD': [1.2710123, 103.82493],
 'CHWEE CHIAN VIEW': [1.279643, 103.7888],
 'WEST COAST CRESCENT': [1.298743, 103.7653247],
 'FABER GREEN': [1.31862, 103.7583806]

In [11]:
def get_lat_long_add(row, lat_long_mapping):
    street = row['street']
    try:
        row['latitude'] = lat_long_mapping[street][0]
        row['longitude'] = lat_long_mapping[street][1]
    except:
        row['latitude'], row['longitude'] = None, None
    return row

In [12]:
new_hdb_df = hdb_df.apply(lambda x: get_lat_long_add(x, lat_long_mapping), axis=1)
new_ura_df = ura_df.apply(lambda x: get_lat_long_add(x, lat_long_mapping), axis=1)

In [13]:
# Only for URA Transactions (PRIVATE)
def get_year(row):
    contract_date = str(row['contractDate'])
    row['year'] = int(contract_date[-2:])
    return row

new_ura_df = new_ura_df.apply(lambda x: get_year(x), axis=1)

In [14]:
final_ura_df = new_ura_df[(new_ura_df.year > 17) & (new_ura_df.latitude.notna()) & (new_ura_df.longitude.notna())]
final_hdb_df = new_hdb_df[(new_hdb_df.latitude.notna()) & (new_hdb_df.longitude.notna())]

# 2. Load Datasets

(1) URA Dataset

In [15]:
ura_5y_trans = final_ura_df
ura_5y_trans["district"] = pd.to_numeric(ura_5y_trans["district"])
print(ura_5y_trans.shape)
ura_5y_trans.head()

(75692, 19)


C:\Users\zywon\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,x,y,nettPrice,latitude,longitude,year
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,20
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,5,Land,Freehold,NaN,NaN,NaN,1.282130,103.786879,18
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,5,Land,Freehold,NaN,NaN,NaN,1.292047,103.768591,20
4,COVE DRIVE,TURQUOISE,CCR,194,01-05,1,1020,3,3180000,Condominium,4,Strata,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244209,103.827487,20


(2) Ameneties Dataset

In [16]:
amenities_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/ameneties_per_district.csv' 
ameneties_per_district = pd.read_csv(amenities_url)
ameneties_per_district.drop(ameneties_per_district.columns[0], axis=1, inplace=True)
print(ameneties_per_district.shape)
ameneties_per_district.head()

(28, 4)


,district,school,hawkercentre,supermarkets
0,1,"['DUNMAN HIGH SCHOOL', 'QUEENSTOWN SECONDARY S...","['Market Street Interim Hawker Centre', 'Maxwe...",[]
1,2,"['DUNMAN HIGH SCHOOL', 'QUEENSTOWN SECONDARY S...",['Tanjong Pagar Plaza Blk 6 (Blk 6 Tanjong Pag...,[]
2,3,"['QUEENSTOWN SECONDARY SCHOOL', 'SCHOOL OF SCI...",['Bukit Merah View Blk 115 (Blk 115 Bukit Mera...,"['FISHOP PTE. LTD.', 'NTUC FAIRPRICE CO-OPERAT..."
3,4,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."
4,5,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."


(3) Bus Stops Dataset

In [17]:
# Obtain bus stop data
bus_stops_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/bus_stops.csv' 
bus_stops = pd.read_csv(bus_stops_url)
bus_stops.drop(bus_stops.columns[0], axis=1, inplace=True)
print(bus_stops.shape)
bus_stops.head()

(5021, 5)


,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414


(4.1) Crime Locations Dataset - Overview

In [18]:
crime_locations_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/overview_years_crimes_locations_v3.csv' 
overview_years_crimes_locations_v3 = pd.read_csv(crime_locations_url)
overview_years_crimes_locations_v3.drop(overview_years_crimes_locations_v3.columns[3], axis=1, inplace=True) # dropped 'Number (Cases)' column coz only difference between this and 'Number' is that 'Number' states the 0s
print(overview_years_crimes_locations_v3.shape)
overview_years_crimes_locations_v3.head()

(1680, 6)


,Year,NPC,Offence,Neighbourhood,Postal,Number
0,2012,Central Police Division - Total,Robbery,[Total]Central Police Division - Total,-,34
1,2012,Central Police Division - Total,Housebreaking,[Total]Central Police Division - Total,-,32
2,2012,Central Police Division - Total,Snatch Theft,[Total]Central Police Division - Total,-,31
3,2012,Central Police Division - Total,Theft Of Motor Vehicle,[Total]Central Police Division - Total,-,47
4,2012,Central Police Division - Total,Outrage Of Modesty,[Total]Central Police Division - Total,-,193


(4.2) Crime Locations Dataset - Average

In [19]:
avg_crime_locations_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/average_crimes_by_location_v3.csv' 
average_crimes_by_location_v3 = pd.read_csv(avg_crime_locations_url)
average_crimes_by_location_v3

,Neighbourhood,Postal,Offence,Number
0,Ang Mo Kio North,569784,Housebreaking,4.857143
1,Ang Mo Kio North,569784,Outrage Of Modesty,26.000000
2,Ang Mo Kio North,569784,Robbery,2.000000
3,Ang Mo Kio North,569784,Snatch Theft,2.714286
4,Ang Mo Kio North,569784,Theft Of Motor Vehicle,5.000000
...,...,...,...,...
210,[Total]Woodlands Police Division - Total,-,Housebreaking,0.285714
211,[Total]Woodlands Police Division - Total,-,Outrage Of Modesty,2.142857
212,[Total]Woodlands Police Division - Total,-,Robbery,0.000000
213,[Total]Woodlands Police Division - Total,-,Snatch Theft,0.000000


(5) Resale Flat Prices Dataset

In [20]:
resale_flat_prices = final_hdb_df
print(resale_flat_prices.shape)
resale_flat_prices.head()

(92235, 14)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,ANG MO KIO AVE 10,1.367602,103.856844
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,ANG MO KIO AVE 4,1.382319,103.838706
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,ANG MO KIO AVE 5,1.376825,103.847556
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,ANG MO KIO AVE 10,1.367602,103.856844
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,ANG MO KIO AVE 5,1.376825,103.847556


In [21]:
sentiment_url = 'https://raw.githubusercontent.com/nicolepng/BT4222/main/Data/combined_sentiment.csv' 
sentiment = pd.read_csv(sentiment_url)
sentiment = sentiment[['district_num','hwz_sentiment','year']]
sentiment.columns = ['district_num', 'weighted_sentiment', 'year']
sentiment

,district_num,weighted_sentiment,year
0,21,0.000000,2018
1,1,0.082516,2018
2,2,0.317045,2018
3,3,0.123678,2018
4,4,0.093259,2018
...,...,...,...
107,24,0.150795,2021
108,25,0.088860,2021
109,26,-0.025054,2021
110,27,0.084183,2021


# 3 BIG DATA SETS

## 3.1. URA
    - ameneties_in_district = ura_5y_trans + ameneties_per_district
    - ameneties_in_district = ameneties_in_district + bus_stops
    - ameneties_and_crimes_in_district = ameneties_in_district + average_crimes_by_location_v3
            
## 3.2. RESALE FLAT PRICES
    - resale_flats_in_district = resale_flat_prices + ameneties_per_district
    - resale_flats_in_district = resale_flats_in_district + bus_stops 
    - resale_flats_and_crime_in_district = resale_flats_in_district + average_crimes_by_location_v3
            
## Append resale_flats_and_crime_in_district to ameneties_and_crimes_in_district
    1. district
    2. street
    3. propertyType (add [propertyType]=Flat in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number

## 3.1 Combine URA and ameneties

In [22]:
# combine ura_5y_trans and ameneties_per_district
ameneties_in_district = ura_5y_trans.join(ameneties_per_district.set_index('district'), on='district')
print(ameneties_in_district.shape)
ameneties_in_district.head()

(75692, 22)


,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,tenure,x,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,...,Freehold,NaN,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,...,Freehold,NaN,NaN,NaN,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR..."
4,COVE DRIVE,TURQUOISE,CCR,194,01-05,1,1020,3,3180000,Condominium,...,99 yrs lease commencing from 2007,28382.47067,25008.33592,NaN,1.244209,103.827487,20,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI..."


## 3.1.1 Combine Combined URA (ameneties_in_district) and Bus Stops

Calculate number of bus stops near each listing

In [23]:
# Formula to calculate distance 
from math import cos, sqrt
R = 6371000 #radius of the Earth in m
def distance(lon1, lat1, lon2, lat2):
    x = (lon2 - lon1) * cos(0.5*(lat2+lat1))
    y = (lat2 - lat1)
    return R * sqrt( x*x + y*y )

In [24]:
busStops = bus_stops.to_dict(orient='records')

In [25]:
# threshold of within 1km
numOfStops = []
busStop_names = [] # to check
for long, lat in zip(ameneties_in_district.longitude, ameneties_in_district.latitude):
    buslist = list(filter(lambda d: distance(d["Longitude"], d["Latitude"], long, lat) <= 1000, busStops))
    busStop_names.append(buslist) # to check
    numOfStops.append(len(buslist))

In [26]:
ameneties_in_district['Bus Stops Nearby'] = numOfStops
ameneties_in_district['Bus Stop Names'] = busStop_names

In [27]:
ameneties_in_district.head()

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,y,nettPrice,latitude,longitude,year,school,hawkercentre,supermarkets,Bus Stops Nearby,Bus Stop Names
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,...,NaN,NaN,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,...,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,...,NaN,NaN,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[]
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,...,NaN,NaN,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,"[{'BusStopCode': 16081, 'RoadName': 'Pasir Pan..."
4,COVE DRIVE,TURQUOISE,CCR,194,01-05,1,1020,3,3180000,Condominium,...,25008.33592,NaN,1.244209,103.827487,20,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,[]


In [28]:
# check bus stops: Neo Pee Teck Lane
print("longitude: ",ameneties_in_district['longitude'][3])
print("latitude: ",ameneties_in_district['latitude'][3])
print("no. bus stops: ",ameneties_in_district['Bus Stops Nearby'][3])
print("name of bus stops: ",ameneties_in_district['Bus Stop Names'][3])

print()
bus_stops

longitude:  103.7685915
latitude:  1.2920468
no. bus stops:  2
name of bus stops:  [{'BusStopCode': 16081, 'RoadName': 'Pasir Panjang Rd', 'Description': 'Aft Palm Green Condo', 'Latitude': 1.29195777797363, 'Longitude': 103.76866138900289}, {'BusStopCode': 16089, 'RoadName': 'Pasir Panjang Rd', 'Description': 'Aft Clementi Rd', 'Latitude': 1.29201138346787, 'Longitude': 103.7690842071612}]



,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414
...,...,...,...,...,...
5016,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5017,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5018,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5019,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [29]:
# check bus stops: Yishun Central 1
print("longitude: ",ameneties_in_district['longitude'][73170])
print("latitude: ",ameneties_in_district['latitude'][73170])
print("no. bus stops: ",ameneties_in_district['Bus Stops Nearby'][73170])
print("name of bus stops: ",ameneties_in_district['Bus Stop Names'][73170])

print()
bus_stops

longitude:  103.8848358
latitude:  1.389954
no. bus stops:  1
name of bus stops:  [{'BusStopCode': 67329, 'RoadName': 'Sengkang East Ave', 'Description': 'Opp Blk 310B', 'Latitude': 1.39001974230515, 'Longitude': 103.8846016818356}]



,BusStopCode,RoadName,Description,Latitude,Longitude
0,481,Woodlands Rd,BT PANJANG TEMP BUS PK,1.383764,103.758300
1,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
2,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
3,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
4,1029,Nth Bridge Rd,Cosmic Insurance Bldg,1.296673,103.854414
...,...,...,...,...,...
5016,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5017,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5018,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5019,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


## 3.1.2 Combine Combined URA (ameneties_in_district) and average overall crime rates 

Average Crime Rates Method
- Using the average crime rates data, find the average overall crime rates for each district
- Average overall crime rates refer to the sum of the average crime rates for all categories

In [32]:
# MAP AVERAGE_CRIMES_BY_LOCATION_V3 [POSTAL] COLUMN INTO A NEW COLUMN NAMED [DISTRICT]

# swap dictionary mapping direction
postal_district = {k: oldk for oldk, oldv in district_postal.items() for k in oldv}

# create new column to obtain the 1st 2 characters of [Postal]
average_crimes_by_location_v3['postal prefix'] = average_crimes_by_location_v3['Postal'].astype(str).str[0:2]

# map postal to district code
average_crimes_by_location_v3['district']= average_crimes_by_location_v3['postal prefix'].map(postal_district) 

# get overall crime rate in each district
average_crimes_by_location_v3 = average_crimes_by_location_v3.groupby(['district']).agg({'Number':'sum'}).reset_index()
average_crimes_by_location_v3.head()

,district,Number
0,10,35.428571
1,12,60.857143
2,14,167.714286
3,15,45.428571
4,16,130.000000


In [33]:
average_crimes_by_location_v3

,district,Number
0,10,35.428571
1,12,60.857143
2,14,167.714286
3,15,45.428571
4,16,130.000000
5,18,163.857143
6,19,202.142857
7,2,200.000000
8,20,122.714286
9,22,226.000000


In [34]:
# convert str to numpy
district_int = average_crimes_by_location_v3.district.astype(int)
district_int = pd.DataFrame(district_int)
average_crimes_by_location_v3['district'] = district_int

In [35]:
# combine ura_5y_trans and ameneties_per_district
ameneties_and_crime_in_district = ameneties_in_district.join(average_crimes_by_location_v3.set_index('district'), on='district')
ameneties_and_crime_in_district = ameneties_and_crime_in_district.reset_index(drop=True)
print(ameneties_and_crime_in_district.shape)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(ameneties_and_crime_in_district.head())

(75692, 25)
              street                     project marketSegment   area  \
0       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR  524.3   
1       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR    308   
2       ZEHNDER ROAD  LANDED HOUSING DEVELOPMENT           RCR    314   
3  NEO PEE TECK LANE  LANDED HOUSING DEVELOPMENT           RCR  159.3   
4         COVE DRIVE                   TURQUOISE           CCR    194   

  floorRange noOfUnits contractDate typeOfSale    price   propertyType  \
0          -         1         0220          3  5500000  Semi-detached   
1          -         1         0918          3  5000000  Semi-detached   
2          -         1         0618          3  4750000  Semi-detached   
3          -         1         0320          3  2630000        Terrace   
4      01-05         1         1020          3  3180000    Condominium   

   district typeOfArea                             tenure            x  \
0         5       Land        

## 3.1.3 Compute remaining lease using tenure
- num of years used = now - completion date
- remaining = 99 - num of years used

In [36]:
contractDate_datetime = []
for index in range(len(ameneties_and_crime_in_district)):
    contractdate_text = str(ameneties_and_crime_in_district['contractDate'][index])
    contractdate_month = int(contractdate_text[0:-2])
    contractdate_year = int("20" + contractdate_text[-2:])
    contractdate_datetime = datetime.date(contractdate_year,contractdate_month,1)
    contractDate_datetime.append(contractdate_datetime)

In [37]:
remaining_lease_list = []
for index in range(len(ameneties_and_crime_in_district)):
    tenure_text = str(ameneties_and_crime_in_district['tenure'][index])
    if tenure_text == "Freehold":
        remaining_lease = "Freehold"
    elif tenure_text == "NA":
        remaining_lease = "NA"
    elif tenure_text == "99 years leasehold":
        remaining_lease = "99 years leasehold"
    else:
        tenure_text_split = tenure_text.split()
        num_years = int(tenure_text_split[0])
        last_text = tenure_text_split[len(tenure_text_split)-1]
        
        if last_text == "01/11/2017":
            date_list = last_text.split("/")
            completion_year = int(date_list[2]) 
            completion_month = int(date_list[1])
        else:
            completion_year = int(last_text)
            completion_month = 1 # Assume January
            
        completion_date = datetime.date(completion_year,completion_month,1)
        contract_date = contractDate_datetime[index]
        
        num_months_used = (contract_date.year - completion_date.year) * 12 + (contract_date.month - completion_date.month)
        num_months_available = num_years*12
        
        remaining_lease_months_total = num_months_available-num_months_used
        
        # convert remaining lease in months to years and months
        if remaining_lease_months_total < 12:
            remaining_lease_months = remaining_lease_months_total
            remaining_lease = str(remaining_lease_months) + " months"
        else:
            remaining_lease_years = math.floor(remaining_lease_months_total/12)
            remaining_lease_months = remaining_lease_months_total - remaining_lease_years*12
            remaining_lease = str(remaining_lease_years) + " years " + str(remaining_lease_months) + " months"
            
    remaining_lease_list.append(remaining_lease)

In [38]:
ameneties_and_crime_in_district['remaining_lease'] = remaining_lease_list
ameneties_and_crime_in_district.head()

,street,project,marketSegment,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,...,latitude,longitude,year,school,hawkercentre,supermarkets,Bus Stops Nearby,Bus Stop Names,Number,remaining_lease
0,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,524.3,-,1,0220,3,5500000,Semi-detached,...,1.282130,103.786879,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
1,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,308,-,1,0918,3,5000000,Semi-detached,...,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
2,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,314,-,1,0618,3,4750000,Semi-detached,...,1.282130,103.786879,18,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,[],NaN,Freehold
3,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,159.3,-,1,0320,3,2630000,Terrace,...,1.292047,103.768591,20,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,"[{'BusStopCode': 16081, 'RoadName': 'Pasir Pan...",NaN,Freehold
4,COVE DRIVE,TURQUOISE,CCR,194,01-05,1,1020,3,3180000,Condominium,...,1.244209,103.827487,20,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,[],NaN,85 years 3 months


## 3.1.4 Additional Cleaning of Columns

In [39]:
# Represent remaining_lease column with numerical values
remaining_lease_numerical = []
for i in range(len(ameneties_and_crime_in_district)):
    text = ameneties_and_crime_in_district['remaining_lease'][i]
    if text == "Freehold":
        num_years = float(999)
    else:
        text_list = text.split(' ')
        len_text_list = len(text_list)
        if 'leasehold' in text:
            num_years = float(text_list[0])
        else:
            years = float(text_list[0])
            months = float(text_list[2])
            num_years_norounding = years + months/12
            num_years = float("{:.2f}".format(num_years_norounding))        
    remaining_lease_numerical.append(num_years)
ameneties_and_crime_in_district['remaining_lease'] = remaining_lease_numerical

In [40]:
ameneties_and_crime_in_district['remaining_lease'].value_counts()

999.00     21918
97.50       1736
97.00       1600
97.33       1323
97.83       1061
           ...  
66.67          1
58.67          1
9939.50        1
64.25          1
852.83         1
Name: remaining_lease, Length: 984, dtype: int64

In [41]:
ameneties_and_crime_in_district['price'] = pd.to_numeric(ameneties_and_crime_in_district['price'])
ameneties_and_crime_in_district['noOfUnits'] = pd.to_numeric(ameneties_and_crime_in_district['noOfUnits'])
ameneties_and_crime_in_district['typeOfSale'] = pd.to_numeric(ameneties_and_crime_in_district['typeOfSale'])
ameneties_and_crime_in_district['area'] = ameneties_and_crime_in_district['area'].astype(float)
# Clean up floor range column 
ameneties_and_crime_in_district['floorRange'] = ameneties_and_crime_in_district['floorRange'].map(lambda x: x.lstrip('~'))
# For Listings with > 1 Unit make sure its the average
ameneties_and_crime_in_district['price'] = np.where(ameneties_and_crime_in_district['noOfUnits'] > 1,
                                           ameneties_and_crime_in_district['price'] / ameneties_and_crime_in_district['noOfUnits'],
                                           ameneties_and_crime_in_district['price'])

ameneties_and_crime_in_district['area'] = np.where(ameneties_and_crime_in_district['noOfUnits'] > 1,
                                           ameneties_and_crime_in_district['area'] / ameneties_and_crime_in_district['noOfUnits'],
                                           ameneties_and_crime_in_district['area'])
# Filter only resale and subsale units (2 and 3)
ameneties_and_crime_in_district = ameneties_and_crime_in_district[ameneties_and_crime_in_district['typeOfSale'].isin([2, 3])].reset_index(drop=True)
# rename columns
ameneties_and_crime_in_district = ameneties_and_crime_in_district.rename(columns={"Number":"crime_number", "area": "floor_area_sqm", "floorRange": "floor_range"})

## 3.1.5 Rearrange columns
    1. district
    2. street
    3. propertyType (add [propertyType]=Flat in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number
    11. Latitude
    12. Longitude
    13. area
    14. Floor Range
    15. sentiment

In [42]:
# Combine sentiment and dataset
ameneties_and_crime_in_district.year = ameneties_and_crime_in_district.year + 2000

ameneties_and_crime_in_district['disyear'] = ameneties_and_crime_in_district['district'].astype(str) + "_" + ameneties_and_crime_in_district['year'].astype(str)

sentiment['disyear'] = sentiment['district_num'].astype(str) + '_' + sentiment['year'].astype(str)

ameneties_and_crime_in_district['sentiment'] = ameneties_and_crime_in_district['disyear'].map(sentiment.set_index('disyear')['weighted_sentiment'])

In [43]:
ameneties_and_crime_in_district.isnull().any()

street              False
project             False
marketSegment       False
floor_area_sqm      False
floor_range         False
noOfUnits           False
contractDate        False
typeOfSale          False
price               False
propertyType        False
district            False
typeOfArea          False
tenure              False
x                    True
y                    True
nettPrice            True
latitude            False
longitude           False
year                False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
Bus Stop Names      False
crime_number         True
remaining_lease     False
disyear             False
sentiment           False
dtype: bool

In [44]:
# drop irrelevant columns
ameneties_and_crime_in_district.columns

Index(['street', 'project', 'marketSegment', 'floor_area_sqm', 'floor_range',
       'noOfUnits', 'contractDate', 'typeOfSale', 'price', 'propertyType',
       'district', 'typeOfArea', 'tenure', 'x', 'y', 'nettPrice', 'latitude',
       'longitude', 'year', 'school', 'hawkercentre', 'supermarkets',
       'Bus Stops Nearby', 'Bus Stop Names', 'crime_number', 'remaining_lease',
       'disyear', 'sentiment'],
      dtype='object')

In [45]:
ameneties_and_crime_in_district = ameneties_and_crime_in_district[['district','street','propertyType','remaining_lease','price',
                                                                   'school','hawkercentre','supermarkets',
                                                                   'Bus Stops Nearby','crime_number', 'latitude', 'longitude',
                                                                   'floor_area_sqm', 'floor_range', 'sentiment']]
ameneties_and_crime_in_district.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,524.3,-,0.154684
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,308.0,-,0.122456
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,314.0,-,0.122456
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,NaN,1.292047,103.768591,159.3,-,0.154684
4,4,COVE DRIVE,Condominium,85.25,3180000.0,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,NaN,1.244209,103.827487,194.0,01-05,0.128195


## 3.2 Combine resale_flat_prices and amenities

In [46]:
# this dictionary shows the mapping from district > 1st 2 characters of postal
district_town = {
  '01': ['Raffles Place', 'Cecil', 'Marina', 'People’s Park'],
  '02': ['Anson', 'Tanjong Pagar'],
  '03': ['Queenstown', 'Tiong Bahru'],
  '04': ['Telok Blangah', 'Harbourfront'],
  '05': ['Pasir Panjang', 'Hong Leong Garden', 'Clementi New Town'], 
  '06': ['High Street', 'Beach Road'],
  '07': ['Middle Road', 'Golden Mile'],
  '08': ['Little India'],
  '09': ['Orchard', 'Cairnhill', 'River Valley'],
  '10': ['Ardmore', 'Bukit Timah', 'Holland Road', 'Tanglin'],
  '11': ['Watten Estate', 'Novena', 'Thomson'],
  '12': ['Balestier', 'Toa Payoh','Serangoon'],
  '13': ['Macpherson', 'Braddell'],
  '14': ['Geylang', 'Eunos'],
  '15': ['Katong', 'Joo Chiat', 'Amber Road'],
  '16': ['Bedok', 'Upper East Coast', 'Eastwood', 'Kew Drive'],
  '17': ['Loyang', 'Changi'],
  '18': ['Tampines', 'Pasir Ris'],
  '19': ['Serangoon Garden', 'Hougang', 'Punggol'],
  '20': ['Bishan', 'Ang Mo Kio'],
  '21': ['Upper Bukit Timah', 'Clementi Park', 'Ulu Pandan'],
  '22': ['Jurong'],
  '23': ['Hillview', 'Dairy Farm', 'Bukit Panjang', 'Choa Chu Kang'],
  '24': ['Lim Chu Kang', 'Tengah'],
  '25': ['Kranji', 'Woodgrove'],
  '26': ['Upper Thomson', 'Springleaf'],
  '27': ['Yishun', 'Sembawang'],
  '28': ['Seletar']
}

# convert all values to upper case
district_town = {key: [ele.upper() for ele in district_town[key] ] for key in district_town }

# swap dictionary mapping direction
town_district = {k: oldk for oldk, oldv in district_town.items() for k in oldv}

# map town to district code
resale_flat_prices['town'].str.upper()  # convert and ensure all town string in the data are in upper case 
resale_flat_prices['district'] = resale_flat_prices['town'].map(town_district)

In [47]:
def get_postal_onemap(place, district_postal):
    start_code= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ str("Ang Mo Kio Ave 5")
    s_response = requests.get(start_code)
    s_data = json.loads(s_response.text)
    postal = None
    count = 0
    while postal is None:
        count += 1
        if count == 10:
            postal = None
            break
        for i in range(len(s_data['results'])):
            postal = s_data['results'][i]['POSTAL']
            try:
                postal = int(postal)
            except:
                continue
    for district, sub_dist in district_postal.items():
        if str(postal)[:2] in sub_dist:
            dist = district
    try:
        return dist
    except:
        return -1

In [48]:
# convert nan to -1
resale_flat_prices['district'] = resale_flat_prices['district'].replace(np.nan, -1)
resale_flat_prices.district.unique()

resale_flat_prices['district'] = np.where(resale_flat_prices['district'] == -1,
                                           get_postal_onemap(resale_flat_prices['district'], district_postal),
                                           resale_flat_prices['district'])
# convert str to numpy
resale_district_int = resale_flat_prices.district.astype(int)
resale_district_int = pd.DataFrame(resale_district_int)
resale_flat_prices['district'] = resale_district_int

In [49]:
# Represent remaining_lease column with numerical values
remaining_lease_numerical = []
resale_flat_prices = resale_flat_prices.reset_index(drop=True)
for i in range(len(resale_flat_prices)):
    text = resale_flat_prices['remaining_lease'][i]
    if text == "Freehold":
        num_years = float('999')
    else:
        text_list = text.split(' ')
        len_text_list = len(text_list)
        if 'leasehold' in text:
            num_years = float(text_list[0])
        else:
            years = float(text_list[0])
            if len(text_list) == 4:
                months = float(text_list[2])
            else:
                months = 0
            num_years_norounding = years + months/12
            num_years = float("{:.2f}".format(num_years_norounding))        
    remaining_lease_numerical.append(num_years)
resale_flat_prices['remaining_lease'] = remaining_lease_numerical

In [50]:
# combine resale_flat_prices and ameneties_per_district
resale_flats_in_district = resale_flat_prices.join(ameneties_per_district.set_index('district'), on='district')
print(resale_flats_in_district.shape)
resale_flats_in_district.head()

(92235, 18)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P..."


## 3.2.1 Combine Combined resale_flat_prices (resale_flats_in_district) and Bus Stops

In [51]:
# threshold of within 1km
numOfStops = []
for long, lat in zip(resale_flats_in_district.longitude, resale_flats_in_district.latitude):
    buslist = list(filter(lambda d: distance(d["Longitude"], d["Latitude"], long, lat) <= 1000, busStops))
    numOfStops.append(len(buslist))

resale_flats_in_district['Bus Stops Nearby'] = numOfStops
resale_flats_in_district.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets,Bus Stops Nearby
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0


## 3.2.2 Combine Combined resale_flat_prices (resale_flats_in_district) and average overall crime rates

In [52]:
# combine ura_5y_trans and ameneties_per_district
resale_flats_and_crime_in_district = resale_flats_in_district.join(average_crimes_by_location_v3.set_index('district'), on='district')
print(resale_flats_and_crime_in_district.shape)
resale_flats_and_crime_in_district.head()

(92235, 20)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,street,latitude,longitude,district,school,hawkercentre,supermarkets,Bus Stops Nearby,Number
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61.33,232000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60.58,250000.0,ANG MO KIO AVE 4,1.382319,103.838706,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,262000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62.08,265000.0,ANG MO KIO AVE 10,1.367602,103.856844,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62.42,265000.0,ANG MO KIO AVE 5,1.376825,103.847556,20,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286


## 3.2.3 Rearrange columns
    1. district
    2. street
    3. propertyType (add [propertyType]=flat_type in resale_flat_prices)
    4. remaining_lease 
    5. price (rename resale_flat_prices[resale_price] as [price]])
    6. school
    7. hawkercentre
    8. supermarkets
    9. Bus Stops Nearby
    10. Crime Number
    11. sentiment

In [53]:
def extract_year(x):
    splitted = x.split('-')
    return splitted[0]

resale_flats_and_crime_in_district['year'] = resale_flats_and_crime_in_district.month.apply(extract_year)

resale_flats_and_crime_in_district['disyear'] = resale_flats_and_crime_in_district['district'].astype(str) + '_' + resale_flats_and_crime_in_district['year'].astype(str)

In [54]:
avg_score = ['10_2017', '12_2017', '14_2017', '16_2017', '18_2017', '19_2017','20_2017',
 '23_2017', '27_2017', '3_2017']

In [55]:
for value in avg_score:
    if value == '3_2017':
        curr_district = value[0]
    else:
        curr_district = value[0:2]
    
    next_value = curr_district + '_2018'
    curr_sentiment = sentiment[sentiment.disyear == next_value]['weighted_sentiment'].item()
    
    sentiment = sentiment.append({'district_num': curr_district, 'disyear': value,
                                 'hwz_comment_count': None,'hwz_sentiment': None,
                                 'reddit_comment_count': None, 'reddit_sentiment': None,
                                 'total_count': None, 'weighted_sentiment': curr_sentiment,
                                 'year': 2017}, ignore_index=True)

In [56]:
resale_flats_and_crime_in_district['sentiment'] = resale_flats_and_crime_in_district['disyear'].map(sentiment.set_index('disyear')['weighted_sentiment'])

In [57]:
resale_flats_and_crime_in_district.isnull().any()

month                  False
town                   False
flat_type              False
block                  False
street_name            False
storey_range           False
floor_area_sqm         False
flat_model             False
lease_commence_date    False
remaining_lease        False
resale_price           False
street                 False
latitude               False
longitude              False
district               False
school                 False
hawkercentre           False
supermarkets           False
Bus Stops Nearby       False
Number                 False
year                   False
disyear                False
sentiment              False
dtype: bool

In [58]:
# drop irrelevant columns
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.drop(['month', 'town','block', 'street_name', 'flat_model', 'lease_commence_date'], axis = 1) 
resale_flats_and_crime_in_district.columns

Index(['flat_type', 'storey_range', 'floor_area_sqm', 'remaining_lease',
       'resale_price', 'street', 'latitude', 'longitude', 'district', 'school',
       'hawkercentre', 'supermarkets', 'Bus Stops Nearby', 'Number', 'year',
       'disyear', 'sentiment'],
      dtype='object')

In [59]:
# add property type
resale_flats_and_crime_in_district['propertyType'] = resale_flats_and_crime_in_district['flat_type']
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.drop(['flat_type'], axis = 1)

# rename resale price as price, Number as crime_number
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.rename(columns={"resale_price": "price","Number":"crime_number", "storey_range": "floor_range"})

In [60]:
resale_flats_and_crime_in_district = resale_flats_and_crime_in_district[['district','street','propertyType','remaining_lease','price',
                                                                         'school','hawkercentre','supermarkets',
                                                                         'Bus Stops Nearby','crime_number', 'latitude',
                                                                         'longitude', 'floor_area_sqm', 'floor_range', 'sentiment']]
resale_flats_and_crime_in_district.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,20,ANG MO KIO AVE 10,2 ROOM,61.33,232000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.367602,103.856844,44.0,10 TO 12,0.15004
1,20,ANG MO KIO AVE 4,3 ROOM,60.58,250000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.382319,103.838706,67.0,01 TO 03,0.15004
2,20,ANG MO KIO AVE 5,3 ROOM,62.42,262000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.376825,103.847556,67.0,01 TO 03,0.15004
3,20,ANG MO KIO AVE 10,3 ROOM,62.08,265000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.367602,103.856844,68.0,04 TO 06,0.15004
4,20,ANG MO KIO AVE 5,3 ROOM,62.42,265000.0,"['BEATTY SECONDARY SCHOOL', 'MAYFLOWER SECONDA...","['Ang Mo Kio Ave 1 Blk 341 (Teck Ghee Court)',...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'G8 MART P...",0,122.714286,1.376825,103.847556,67.0,01 TO 03,0.15004


## 3.2.4 Align HDB Floors to URA Floors

In [61]:
# Align HDB Floor to Same as URA Data

replace_floors = {
    '01 TO 03' : '01-05',
    '04 TO 06' : '01-05',
    '07 TO 09' : '06-10',
    '10 TO 12' : '11-15',
    '13 TO 15' : '11-15',
    '16 TO 18' : '16-20',
    '19 TO 21' : '16-20',
    '22 TO 24' : '21-25',
    '25 TO 27' : '26-30',
    '28 TO 30' : '26-30',
    '31 TO 33' : '31-35',
    '34 TO 36' : '31-35',
    '37 TO 39' : '36-40',
    '40 TO 42' : '41-45',
    '43 TO 45' : '41-45',
    '46 TO 48' : '46-50',
    
}

resale_flats_and_crime_in_district = resale_flats_and_crime_in_district.replace({'floor_range': replace_floors})

## 3.3 Combine ameneties_and_crime_in_district and resale_flats_and_crime_in_district

In [121]:
df = ameneties_and_crime_in_district.append(resale_flats_and_crime_in_district)
df.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,524.3,-,0.154684
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,308.0,-,0.122456
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",0,NaN,1.282130,103.786879,314.0,-,0.122456
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,"['HILLGROVE SECONDARY SCHOOL', 'QUEENSTOWN SEC...","['Pasir Panjang Food Centre', 'Clementi West S...","['NTUC FAIRPRICE CO-OPERATIVE LTD', 'COLD STOR...",2,NaN,1.292047,103.768591,159.3,-,0.154684
4,4,COVE DRIVE,Condominium,85.25,3180000.0,"['FAIRFIELD METHODIST SCHOOL (SECONDARY)', 'QU...",['Telok Blangah Drive Blk 82 (Telok Blangah Ma...,"['NTUC FAIRPRICE CO-OPERATIVE LTD', 'THE QUALI...",0,NaN,1.244209,103.827487,194.0,01-05,0.128195


In [122]:
df.shape

(133690, 15)

In [123]:
df.columns

Index(['district', 'street', 'propertyType', 'remaining_lease', 'price',
       'school', 'hawkercentre', 'supermarkets', 'Bus Stops Nearby',
       'crime_number', 'latitude', 'longitude', 'floor_area_sqm',
       'floor_range', 'sentiment'],
      dtype='object')

In [124]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number         True
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

## 4. Further cleaning of data

In [126]:
# convert nan to -1
df['crime_number'] = df['crime_number'].replace(np.nan, -1)
#df['crime_number'] = df['crime_number'].fillna((df['crime_number'].mean()))
df.crime_number.unique()

array([ -1.        , 193.28571429, 200.        ,  35.42857143,
       113.        , 190.71428571, 167.71428571,  60.85714286,
        45.42857143, 202.14285714, 130.        , 122.71428571,
       163.85714286, 169.42857143, 226.        ,  64.        ,
        78.92857143])

In [127]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

In [128]:
# Convert list of schools, hawkers and supermarkets into num of each facility around
# School

import json

num_sch = []
for sch in df['school']:
    curr = sch.strip('][').split(',') 
    if " SINGAPORE'" in curr:
        curr.remove(" SINGAPORE'")
    num_sch.append(len(curr))

df['school'] = num_sch

# Hawker Centre
num_hawker = []
for hawker in df['hawkercentre']:
    curr = hawker.strip('][').split(',')
    num_hawker.append(len(curr))

df['hawkercentre'] = num_hawker

# Supermarkets
num_supermarkets = []
for supermarket in df['supermarkets']:
    curr = supermarket.strip('][').split(',')
    num_supermarkets.append(len(curr))

df['supermarkets'] = num_supermarkets

df

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,27,6,8,0,-1.0,1.282130,103.786879,524.3,-,0.154684
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,27,6,8,0,-1.0,1.282130,103.786879,308.0,-,0.122456
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,27,6,8,0,-1.0,1.282130,103.786879,314.0,-,0.122456
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,27,6,8,2,-1.0,1.292047,103.768591,159.3,-,0.154684
4,4,COVE DRIVE,Condominium,85.25,3180000.0,11,3,2,0,-1.0,1.244209,103.827487,194.0,01-05,0.128195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92230,27,YISHUN AVE 6,EXECUTIVE,66.67,655000.0,18,2,19,0,64.0,1.426142,103.849653,148.0,11-15,0.084183
92231,27,YISHUN RING ROAD,EXECUTIVE,66.83,650000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,11-15,0.084183
92232,27,YISHUN RING ROAD,EXECUTIVE,66.17,607000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,01-05,0.084183
92233,27,YISHUN ST 81,EXECUTIVE,66.00,670000.0,18,2,19,0,64.0,1.415253,103.837021,142.0,01-05,0.084183


In [129]:
df.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
price               False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

## 5. Normalise Data and Split Data

In [102]:
df

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,ZEHNDER ROAD,Semi-detached,999.00,5500000.0,27,6,8,0,-1.0,1.282130,103.786879,524.3,-,0.154684
1,5,ZEHNDER ROAD,Semi-detached,999.00,5000000.0,27,6,8,0,-1.0,1.282130,103.786879,308.0,-,0.122456
2,5,ZEHNDER ROAD,Semi-detached,999.00,4750000.0,27,6,8,0,-1.0,1.282130,103.786879,314.0,-,0.122456
3,5,NEO PEE TECK LANE,Terrace,999.00,2630000.0,27,6,8,2,-1.0,1.292047,103.768591,159.3,-,0.154684
4,4,COVE DRIVE,Condominium,85.25,3180000.0,11,3,2,0,-1.0,1.244209,103.827487,194.0,01-05,0.128195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92230,27,YISHUN AVE 6,EXECUTIVE,66.67,655000.0,18,2,19,0,64.0,1.426142,103.849653,148.0,11-15,0.084183
92231,27,YISHUN RING ROAD,EXECUTIVE,66.83,650000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,11-15,0.084183
92232,27,YISHUN RING ROAD,EXECUTIVE,66.17,607000.0,18,2,19,0,64.0,1.424340,103.845070,146.0,01-05,0.084183
92233,27,YISHUN ST 81,EXECUTIVE,66.00,670000.0,18,2,19,0,64.0,1.415253,103.837021,142.0,01-05,0.084183


In [130]:
# Label Encoding 
df['street'] = df['street'].astype('category')
street_dict = dict(zip(df['street'].cat.codes, df['street']))
df['street'] = df['street'].cat.codes

df['propertyType'] = df['propertyType'].astype('category')
property_dict = dict(zip(df['propertyType'].cat.codes, df['propertyType']))
df['propertyType'] = df['propertyType'].cat.codes

df['floor_range'] = df['floor_range'].astype('category')
floor_dict = dict(zip(df['floor_range'].cat.codes, df['floor_range']))
df['floor_range'] = df['floor_range'].cat.codes

In [140]:
# Save as pickle to use later in models as mapping dict
with open('street_mapping.pickle', 'wb') as handle:
    pickle.dump(street_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('property_mapping.pickle', 'wb') as handle:
    pickle.dump(property_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('floor_mapping.pickle', 'wb') as handle:
    pickle.dump(floor_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [132]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'], 
                                                    test_size=0.1,
                                                    random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120321, 14)
(13369, 14)
(120321,)
(13369,)


In [76]:
print(X_train.shape)
print(X_train.isnull().any())
print()
print(X_test.shape)
print(X_test.isnull().any())
print()
print(y_train.shape)
print(y_train.isnull().any())
print()
print(y_test.shape)
print(y_test.isnull().any())
print()

(120135, 14)
district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

(13349, 14)
district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

(120135, 1)
price    False
dtype: bool

(13349, 1)
price    False
dtype: bool



## 6. Export to csv

In [77]:
# X_train.to_csv("/Users/vickiyew/Documents/BT4222/project/data/training_set.csv", index = False)
# X_test.to_csv("/Users/vickiyew/Documents/BT4222/project/data/testing_set.csv", index = False)
# y_train.to_csv("/Users/vickiyew/Documents/BT4222/project/data/training_labels.csv", index = False)
# y_test.to_csv("/Users/vickiyew/Documents/BT4222/project/data/testing_labels.csv", index = False)
# df.to_csv("/Users/vickiyew/Documents/BT4222/project/data/combined.csv", index = False)

In [136]:
X_train.to_csv("training_set.csv", index = False)
X_test.to_csv("testing_set.csv", index = False)
y_train.to_csv("training_labels.csv", index = False)
y_test.to_csv("testing_labels.csv", index = False)

In [78]:
#df.to_csv("/Users/admin/Desktop/Y3S2/BT4222/combined.csv", index = False)

In [79]:
#df.to_csv('combined.csv')